(cont:mm:overview)=
# Overview of MM

Today the dominant type of memory management that is widely assumed and used is paged virtual memory management.  In this chapter we describe the evolution of memory management from [simple](cont:mm:overview:simple) physical, to segmented virtual, to today's paged virtual systems. 


```{figure} ../images/mm/phys-mem.png
---
width: 80% 
name: mem-pys-mem-fig
---
Some description of this figure. 
```


(cont:mm:overview:simple)=
## Simple physical memory management.
From a hardware perspective, the simplest memory management model is to have processes just use the physical address space; where instructions executed on the processor directly reference physical memory addresses.  For example, if the application wants to load a register, it would issue the instruction directly indicating the physical memory address.  Similarly, every branch and procedure call directly uses physical memory addresses.  

As shown in {numref}`mem-pys-mem-fig`.

Early in the development of computer hardware and operating systems physical memory management was used.  While this is very primitive, physical memory management is still used in simple embedded systems.

Physical memory management two major problems: 1) there is no protection between processes, and 2) there is the need for relocation. 

without any translation. This implies processes sharing that physical address space are randomly scattered around within it.  Each process also must deal with different addresses and there is no protection between those processes.  
  
### Sharing one physical address space.


Users were typically limited to trusted colleges and few users shared the computer at the same time which was good because sharing one physical address space offers no protection between users, one program running can simply read and write all the memory regardless of who was using it.  Over time the quantity of memory expanded allowing more users to run programs on the computer at the same time.  In addition to a lack of protection sharing a single physical address space requires every program to deal with different physical addresses since its not know exactly where they will be loaded and run.  Each time a program is run its likely to loaded and run at a different address.  Dealing with this imposes a significant burden on basic programming.  To eliminate this burden operating system designers developed a relocation technique that corrected all the physical addresses when a program was loaded as a specific address. 

### Relocating programs.
When a program is compiled and linked into object code the address at which the program will run must be known so that branch destinations can be established.  When the program is loaded into memory it must be loaded at the correct address or the program will branch to the wrong locations.  Since more than one program can run at the same time there is no guarantee that the program will be loaded and run at the address it was linked to run.  For this reason programs are usually linked as relocatable.  Relocatable programs have a table of all addresses within the program image that must be changed if it is loaded at a different address than it was linked before it can be run.   This is how more that one program can be run at the same time in a system that only supports physical memory management.


## Virtual memory management
As discussed above physical memory management model has significant drawbacks, lack of protection and mandatory relocation being the most significant.  For this reason memory management subsystem has evolved over time give each process its own private address space known as a virtual address space.  The virtual address space for each process looks identical, that is starting at zero going up to some limit and provides protection from other virtual address spaces so that one process can not corrupt any other processes.

## Segmentation.
In the simplest form of segmentation the hardware provides 2 registers that are loaded each time a process acquire the CPU, a base register and a limit register.  For a given process the base register contains the physical address that the program was loaded at and the limit register contains the size of the program that was loaded into memory.  Every process has a virtual address space starting at zero and a size determined by the actual program size that the process is running.  The process specific base and limit registers which are loaded every time a process acquires the CPU establishes the bounds of the virtual address space for every process.  For each and every memory reference the hardware adds the virtual address to the base register to determine a physical address and insures that the physical address is between the base register and base register plus the limit register.  If it is outside those bounds the program is terminated with an illegal virtual memory reference error.  Segmentation solves both the lack of protection and the mandatory relocation requirements of physical addressing.  Segmentation has little or no performance overhead because the hardware performs the virtual to physical translation or the addition of the virtual address and the base register to determine every physical address.

### Single segments per address space
So far we discussed a segmentation implementation that provides one base register and one limit register in hardware and one of each of those process specific values that gets loaded into those registers when context switching to a given process.  Since there is only one of each register, the entire process virtual address space must be physically contiguous and all text, data and stack must be within that single memory region.  While this is a huge improvement over a physical memory model it limits the size of the virtual address space to being static and not expandable.  There is no way to dynamically increase the size of the text, data or stack regions of a process at run-time, everything must be allocated in advance.  This requires allocating physical memory that might never be used.

### Multiple segments per address space.
As mentioned earlier a single base and limit segment register implies that an entire process virtual address space is a one physically contiguous region of physical memory mapped into one virtually contiguous virtual region of virtual memory.  This means that the text, data and stack regions must be packed tightly together in both physical and virtual memory unless we are willing to waste both physical and virtual memory.  Also, with only one segment register its not possible to offer different types of protections for the various regions of the virtual address space.  In other words all of virtual memory must be readable, writable and executable since data must be both readable and writable and text must be executable.  It would be nice to prevent data regions from being executable and text regions from being readable and writable for security and debug optimizations.

This can be achieved by the hardware implementing multiple segment and limit registers with only specified permissions for text, data and stack regions and having the operating system use those registers when context switching to a process.  When mapping the text into a virtual region the operating system can specify an execute only region that does not have to be adjacent to other non-executable regions.  When mapping data into virtual memory the operating system can specify read/write only thereby preventing execution of data regions.  Finally the stack can also be non-executable but also the operating system can move the virtual memory stack region away from any other region making it easier to debug common programming problems like stack overflows.  Finally multiple segment registers eliminates the necessity for the text, data and stack regions to be physically contiguous.  This allows a program to be split up into multiple smaller regions both physically and virtually making it much easier to hold more programs in physical memory at the same time.

### Private versus Global segments.
Every process virtual address space consists of 2 types of regions, Private and Global.  The private regions for a process are the program specific text, data and stack regions that the process is running.  The global regions include the operating system that is and must be mapped into every process address space.  As we discussed earlier in this course the operating system consists of all the software that executes on behalf of the currently running program as well as basic system overhead that runs on behalf of the system.  This includes all the system calls the operating system supports.  Since every process must map the operating system kernel it is shared between every running process rather than each process containing a separate copy.  The global segment registers are used to map the shared kernel text, data and kernel stack area in every process and at the same virtual addresses.  When a context switch occurs only the private segments registers are changed for the newly running process, there is no need to change the global segment registers since they are identical for every process.
  #### Compaction.
When a new process is created and a program runs the kernel reads the program text, data and stack memory into the available or free physical memory locations.  From there the private segment registers are use to map that physical memory into the private virtual address space of the process, allowing the process to run the program.  When a process exits, the physical memory regions that the process consumed is made available or freed onto a physical memory free list.  Over time as processes are created, run and exit the physical memory becomes more and more fragmented.  After a while as processes come and go its likely that the sum of available physical memory is large enough to satisfy a request but there is no physically contiguous free memory region large enough to hold the request.  When this happens the operating system must move or coalesce the used memory regions together thereby creating a large contiguous available or free region.  Now one or more requests can be satisfied.  This is all made possible because the base registers of the processes that map these moved regions can be updated to the new locations of the physical memory and because the operating system has the ability to relocate programs in physical memory as we discussed in the physical memory management model.
  #### Limitations of segmentation.
Segmentation, especially with multiple segment registers along with private and global segment registers provides a huge benefit over a physical memory management model.  We can now support many processes running at the same time with protection between processes and even protection within a process.  However that is still one major weakness with segmentation, the virtual address size can never exceed the physical address size with segmentation.  It would be very convenient to be able to allocate a very large sparse region of virtual memory and only actually use a small subset of it.  Imagine allocating an array of fixed size records for every possible student at Boston University and indexing that array by the student's social security number.  There are 10^9 or 1 Billion social security numbers but only several thousand students.  Such a large sparse array could not be implemented with segmentation unless the system actually had all the necessary physical memory for every possible social security number.  Imagine being able to map millions of files in a virtual address space on a system that didn't have all that much physical memory.

### Paging
As mentioned earlier, a major limitation of segmentation is not being able to support a virtual address space which is larger than the amount of physical memory on a system.  In order to do this we would have to be able to have portions of the virtual address space that are not backed by RAM and other portions that are.  This is not possible with segmentation because we would always need part of the text, data and stack sections in physical memory and segmentation requires the whole segment in physical memory.  Also segments will always be different sizes, the data section is likely to be much larger than the text section which is likely to be much larger than the stack section.  So, what we really need is a virtual memory model that only requires a random number of small fixed size portions of physical memory to be mapped in virtual memory at a time.

To satisfy these requirements modern systems implement a virtual memory model known as paging.  With paging virtual memory is divide into small fixed size "pages" and physical memory is divided into same fixed size "page frames".  For example the 32-bit Intel x86 system has a 4GB(2^32) virtual memory size, a 4KB(2^12) virtual page size and a 4KB(2^12) physical page size.  in this example each process has a private 4GB(2^32) byte virtual address space divided into 1048576(2^32/2^12 = 2^20) 4KB(2^12) "pages".  The system itself manages physical memory or RAM in a variety of different sizes, typically from 1GB(2^30) up to 4GB(2^32) on the x86 and it is split up into 262144(2^30/2^12 = 2^18) for 1GB of RAM up to 1048576(2^32/2^12 = 2^20) "page frames" of physical memory for 4GB of RAM.

#### Virtual pages and page frames.
As mentioned above both the virtual memory or virtual address space and the physical memory consists of small fixed size regions called pages and page frames respectively.  Pages and page frames must be the same size so that page frames can be mapped into pages and size of pages and page frames are always a power-of-two.  Also as mentioned earlier the x86 and x86_64 Intel systems have a 4KB page size.  Both the virtual address space and the physical memory can be envisioned as an array of pages and page frames respectively.  The virtual address space is always a power-of-two in size but the physical memory does no have to be.  For example the x86 system described earlier is a 32-bit system therefore the size of the process's virtual address space is 2^32 or 4GB, or an array of 1M 4096-byte pages.  The physical memory size can be a power-of-two but it does no have to be.  You could install 1GB of RAM which is 256K 4096-byte page frames but you could also install 1.5GB of RAM which is 384K 4066-byte page frames.

The virtual address size(Vsize) is equal to the page size(Psize) multiplied by the number of pages(Npages) or Vsize = Psize X Npages and all 3 must be powers-of-two.  Calculating the Vsize, Psize or Npages is simple if you know the other 2 especially since they are always powers-of-two.  For example a system that has a 1GB Vsize and 1KB Psize has 1M Npages(1GB Vsize/1KB Psize = 1M Npages) and since they must all be powers-of-two (2^30 - 2^10 = 2^20 which is 1M pages).

#### Page tables.
The data structure that contains the virtual page to physical page frame mapping in a paging system is the page table. There is one page table for each process running in the system.  The page table is an array of page table entries.  There is one page table entry for each page in the virtual address space.  The page table entry is either NULL if no mapping exists or its a page frame number if a mapping does exist.  The number of page table entries in a page table is equal to the number of virtual pages in the process address space.  The Nth entry in the page table describes the virtual to physical mapping for the Nth virtual page in the address space.  In other words pagetable[0] describes the mapping for virtual page 0, pagetable[1] describes the mapping for virtual page 1...pagetable[N] describes the mapping for virtual page N.  Each time the CPU references a virtual address the hardware uses the virtual address to index into the page table and extract the page table entry.  If that page table entry contains a valid page frame number the hardware references the associated physical page.  This is known as virtual to physical translation and while there is some performance consequence the hardware is optimized to process the page table.  If that page table entry is NULL a page fault occurs and the operating system resolves it, much more on this later.

#### Virtual address spaces larger than RAM.
As mentioned earlier the main weakness in segmentation is not being able to have a larger virtual address space size than the physical memory size.  Paging solves that problem is a profound way, not only can a single virtual address space be orders of magnitude larger than the physical memory size, there can practically be an unlimited number of processes or virtual addresses actively running at the same time on a paging system.  Its the page table that makes this possible.

In the x86 example each process has a 4GB Vsize which is 1M virtual pages for each process.  If that x86 system is currently running 1000 processes there are 1G virtual pages on the whole system.  Since the virtual address space on an x86 paging system consists of 1M 4KB pages only a small subset of those virtual pages must map physical pages at any one time.  As long as that subset size is smaller than the number of physical pages that process can run.  If there are 1000 processes that need to run, they can all run as long as the sum of all those subset sizes are smaller than the number of physical pages.

When a child process is created its virtual address is created as well.  Typically that virtual address space is a shared copy of the parent's virtual address space so there is little or no additional physical memory required to get that child process to start running.  Alternately in some operating systems the virtual address space of a newly created process can be empty(windows for example).  This also has little or no additional physical memory requirement to get that process running.  As that process starts running it references virtual pages that are either shared with the parent or have no physical memory mapped.  If there is no physical memory mapped at that virtual address this results in what is know as a page fault(more on this later)  The page fault handling routine allocates a page frame of physical memory and maps it into the virtual page that caused the page fault.  Once the process resumes execution the faulting virtual address now has a page frame mapped into it so the process continues running successfully, once again, much more on page faults later.

#### Management of physical memory.
As we mentioned earlier the physical memory is arranged into fixed size page frames, 4096 bytes or 4KB each is typical.  The kernel maintains a data structure known as the struct_page for each page frame of physical memory.  The kernel also maintains an array of struct_pages called the page_array. 
In the page_array the Nth struct_page in the page_array is used to represent and monitor the Nth page frame of physical memory.  In other words the struct_page at page_array[0] represent the page frame at physical address 0, the struct_page at page_array[1] represent the page frame at physical address 4096, the struct_page at page_array[2] represent the page frame at physical address 8192 and so on.

The kernel also maintains a free list of page_structs representing free pages.  When the kernel boots up every page of memory that is not being used is freed or inserted on the free list.  As memory is needed by the kernel for various reasons(handling page faults for example), pages is removed from the free list or "allocated" and used.  When memory is no longer being used(when a process exits for example), pages are inserted back on the free list or "freed".  As memory is allocated and freed the kernel maintains an accurate free page list count. 

#### Management of virtual memory.
Each process had its own private virtual address space which is a large fixed size based on the number of hardware address bit, for example the x86 32-bit system has a 4GB or 2^32 virtual address space for every process.  In order to maintain this virtual address space the kernel allocates a page table that maintains the mapping for every virtual page.  For example the x86 32-bit system has 1M virtual pages so a 1M entry page table is allocated when a process is created.

The kernel also maintains a list of virtual_memory_area data structures or VMAs representing areas within the virtual address space that are valid.  The VMA struct must include the beginning and ending virtual addresses and protection for the region it represents.  For example when a program is mapped in virtual address space to run separate VMAs will be allocated and added to the list for the text, data and stack sections with appropriate beginning addresses, ending addresses and protections.  The kernel will only allow virtual memory references to locations within a VMA, any memory references outside of a region described by a VMA are considered illegal and will cause the program to terminate.  

Its very common for a virtual address space to be very sparse and out of millions of potential virtual pages to have a few hundred to a few thousand valid ones described by the VMAs.  Its possible for a process and its running program to use the entire virtual address space but not very common at all.

There are 2 basic types of virtual memory regions; anonymous and filebacked.  Anonymous virtual memory is any virtual memory that is not backed by a file on some storrage.  This included stacks, heaps, BSS

#### Over committing physical memory.

#### Paging and Swapping.

    - Quick overview of what memory manager needs to do
        - compaction 
        - bringing things in, demand paging
            swapping, paging
        - deciding what to kick out - reclamation
        - anonymous memory
        - file backed and caching storage
        - example: look at elf, parts of the program...